In [1]:
import pandas as pd
import re
import os
import logging
import numpy as np
from collections import defaultdict

![paper](img/paper_shot.png "Article")

# Bioinformatic pipeline

![PRJNA632856](img/PRJNA632856.png "PRJNA632856 description")

## Download raw files and metadata from NCBI

In [ ]:
! wget 'http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?save=efetch&rettype=runinfo&db=sra&term=PRJNA632856' -O -  > PRJNA632856.cvs

In [ ]:
%%bash
parallel --verbose -j 20 prefetch {} ::: $(awk 'BEGIN { FS="," } !/Run/{print $1}' PRJNA632856.cvs)
parallel --verbose -j 20 fastq-dump --split-files {} ::: $(awk 'BEGIN { FS="," } !/Run/{print $1}' PRJNA632856.cvs)
parallel bgzip {} ::: *.fastq
mkdir RAW && mv *gz RAW/

## Run snakemake to analyze fastq.gz raw files
### Create a config file with raw files path

In [ ]:
path = "RAW"
samples = defaultdict(dict)
seen = set()
for dir_name, sub_dirs, files in os.walk(os.path.abspath(path)):
        for fname in files:
            if ".fastq.gz" in fname:
                sample_id = fname.split(".fastq.gz")[0]
                sample_id = sample_id.replace("_1", "").replace("_2", "")
                fq_path = os.path.join(dir_name, fname)
                if "_1" in fname:
                    samples[sample_id]['R1'] = fq_path
                else:
                    samples[sample_id]['R2'] = fq_path
samples= pd.DataFrame(samples).T
samples.index.name = 'sample_id'
samples.to_csv("config.tsv",sep='\t')

### Run the snakemake pipeline 
Rules used to run the snakemake are synthetizased in the figure below. Results will be stored in the output folder. 


![pipeline](img/pipeline.png "pipeline description")

In [ ]:
# Run the loop
! snakemake --use-conda --conda-frontend mamba -j 15 && snakemake --report report.html 

## Statistical analysis on snakemake results

In [ ]:
## Create a metadata file from metadata csv downloaded from ncbi
S = pd.read_table("PRJNA632856.cvs",sep=",",index_col=0)[["SampleName"]]
S[['SRR','Sample','Replicate','kit']] = S.SampleName.str.replace("_","").str.replace("replicate","Replicate").str.split('(?=[A-Z])',expand=True)
S['experiment'] = S['Sample'].apply(lambda x: 'sample' if 'Patient' in x else 'mock')
S = S[['Sample','Replicate','kit','experiment']]
S = S.sort_values(['experiment','Sample','kit'])